# Review Session 4: Mutable State and References

## What is mutable state

State that is mutable, or changable, such as a variable in Python, or a var in scala.

In [ ]:
// immutable
val s = 5
s = 10 // cannot reassign to a val, it is immutable

In [ ]:
// mutable
var s = 5
s = 10 // can reassign to a var, it is mutable

## Why talk about mutable state

For most of the class our language, Lettuce, has been a *pure* language

![man eating lettuce](https://c.tenor.com/LPwE-EjeFpYAAAAC/vegetarian-lettuce.gif)
$$\begin{array}{rcll}
\mathbf{Program} & \rightarrow & TopLevel(\mathbf{Expr}) \\[5pt]
\mathbf{Expr} & \rightarrow & Const(\mathbf{Number}) \\
 & | & Ident(\mathbf{Identifier}) \\
 & | & Plus(\mathbf{Expr}, \mathbf{Expr}) \\
 & | & Minus(\mathbf{Expr}, \mathbf{Expr}) \\
 & | & Mult (\mathbf{Expr}, \mathbf{Expr}) \\
 & | & Geq (\mathbf{Expr}, \mathbf{Expr}) \\
 & | & Eq (\mathbf{Expr}, \mathbf{Expr}) \\
 & | & IfThenElse(\mathbf{Expr}, \mathbf{Expr}, \mathbf{Expr}) & \text{if (expr) then expr else expr} \\
 & | & Let( \mathbf{Identifier}, \mathbf{Expr}, \mathbf{Expr}) & \text{let identifier = expr in expr} \\
 & | & FunDef( \mathbf{Identifier}, \mathbf{Expr}) & \text{function (identifier-formal-parameter) expr } \\ 
 & | & FunCall(\mathbf{Expr}, \mathbf{Expr}) & \text{function call - expr(expr)} \\
\end{array}$$


This makes reasoning about code easy, as the output of the function should only depend on the input of the function.

However, many languages are impure. Java, Python, C++, Javascript, and even Scala.
This can make reasoning about code more difficult as one might have to also consider the values of variables and the function may have side effects.
Side effects are not necessarily a bad thing and can make code easier to write, but can cause nasty surprises if not properly documented and considered when writing new code.

Regardless, it makes sense for us to discuss mutable state and possible implementations of mutable state.

## Explicit References

In trying to add references, the easiest way is to make everything very explicit: anything dealing with a reference looks like it.

This is similar to how pointers work in languages like C++.

We might extend our basic lettuce grammar as follows:

$$\begin{array}{rcll}
\mathbf{Program} & \rightarrow & TopLevel(\mathbf{Expr}) \\[5pt]
\mathbf{Expr} & \rightarrow & Const(\mathbf{Number}) \\
 & | & Ident(\mathbf{Identifier}) \\
 & | & Plus(\mathbf{Expr}, \mathbf{Expr}) \\
 & | & Minus(\mathbf{Expr}, \mathbf{Expr}) \\
 & | & Mult (\mathbf{Expr}, \mathbf{Expr}) \\
 & | & Geq (\mathbf{Expr}, \mathbf{Expr}) \\
 & | & Eq (\mathbf{Expr}, \mathbf{Expr}) \\
 & | & IfThenElse(\mathbf{Expr}, \mathbf{Expr}, \mathbf{Expr}) & \text{if (expr) then expr else expr} \\
 & | & Let( \mathbf{Identifier}, \mathbf{Expr}, \mathbf{Expr}) & \text{let identifier = expr in expr} \\
 & | & FunDef( \mathbf{Identifier}, \mathbf{Expr}) & \text{function (identifier-formal-parameter) expr } \\ 
 & | & FunCall(\mathbf{Expr}, \mathbf{Expr}) & \text{function call - expr(expr)} \\
 & | & \color{red}{NewRef}(\mathbf{Expr}) & \text{create a new reference and place the value of expr in it.}\\
 & | & \color{red}{DeRef}(\mathbf{Expr}) & \text{dereference a reference and return the value}\\
 & | & \color{red}{AssignRef}(\mathbf{Expr}, \mathbf{Expr}) & \text{assign a reference to a value. }
\end{array}$$

In [ ]:
sealed trait Program
sealed trait Expr

case class TopLevel(e: Expr) extends Program

case class Const(v: Double) extends Expr // Expr -> Const(v)
case class Ident(s: String) extends Expr // Expr -> Ident(s)

// Arithmetic Expressions
case class Plus(e1: Expr, e2: Expr) extends Expr // Expr -> Plus(Expr, Expr)
case class Minus(e1: Expr, e2: Expr) extends Expr // Expr -> Minus(Expr, Expr)
case class Mult(e1: Expr, e2: Expr) extends Expr // Expr -> Mult (Expr, Expr)

// Boolean Expressions
case class Geq(e1: Expr, e2:Expr) extends Expr
case class Eq(e1: Expr, e2: Expr) extends Expr

//If then else
case class IfThenElse(e: Expr, eIf: Expr, eElse: Expr) extends Expr

//Let bindings
case class Let(s: String, defExpr: Expr, bodyExpr: Expr) extends Expr

//Function definition
case class FunDef(param: String, bodyExpr: Expr) extends Expr

// Function call
case class FunCall(funCalled: Expr, argExpr: Expr) extends Expr

// New Ref
case class NewRef(e: Expr) extends Expr

//DeRef
case class DeRef(lval: Expr) extends Expr

//AssignRef
case class AssignRef(lval: Expr, rval: Expr) extends Expr


How do we go about imlementing support for references in our interpreter for lettuce?
We need to make use of a new datastructure called the store.

The store is like a heap in languages like C++, in that we have addresses into the store, and each address in the store has a value associated with it.  We assume we have an infinite amount of space, and let the addresses be the natural numbers, starting at 0.
Then, a reference can be represented as an address into store.
What operations do we need for our store?

How does our eval function need to change? (Recall that the original interpreter takes in a expression and an environment, and returns the value corresponding to the expression)

Our eval function now looks like this:
$$ \mathbf{eval}(\texttt{e}, \text{env}, \text{store}) = (\text{value}, \text{new-store}) $$

Our new inference rules now need to keep track of the side effects that can happen.

$$\newcommand\semRule[3]{\begin{array}{c} #1 \\ \hline #2 \\\end{array}\ \ \text{(#3)}} $$
$$\newcommand\eval{\mathbf{eval}}$$

Some of our rules are almost unchanged:
$$\semRule{}{\eval(\texttt{Const(f)}, \sigma, s) = ( f, s) }{const}$$
$$\semRule{x \in \text{domain}(\sigma)}{\eval(\texttt{Ident(x)}, \sigma, s) = (\sigma(x), s)}{ident-ok}\;\;\;
 \semRule{x \not\in \text{domain}(\sigma)}{\eval(\texttt{Ident(x)}, \sigma, s) = (\mathbf{error}, s)}{ident-nok}\;\;\; $$
 
However, the rules for other expressions have become more complicated.  Take the plus case for example:

$$\semRule{ \eval(\texttt{e1}, \sigma, s) = (v_1, \color{red}{s_1})\;\;\; v_1 \in \mathbb{R}\;\;\; \eval(\texttt{e2}, \sigma, \color{red}{s_1}) = (v_2, \color{blue}{s_2})\;\;\; v_2 \in \mathbb{R} }{ \eval(\texttt{Plus(e1, e2)}, \sigma, s) = (v_1 + v_2 , \color{blue}{s_2})}{plus-ok} $$

Let's break down this inference rule.

We will need to write error handling rules. Here we can resort to short circuiting as before.
$$\semRule{ \eval(\texttt{e1}, \sigma, s) = (v_1, \color{red}{s_1})\;\;\; \color{red}{v_1 \not\in \mathbb{R}}}{ \eval(\texttt{Plus(e1, e2)}, \sigma, s) = (\mathbf{error} , \color{red}{s_1})}{plus-nok-1} \;\;
\semRule{ \eval(\texttt{e1}, \sigma, s) = (v_1, \color{red}{s_1})\;\;\; v_1 \in \mathbb{R}\;\;\; \eval(\texttt{e2}, \sigma, \color{red}{s_1}) = (v_2, \color{blue}{s_2})\;\;\; \color{red}{v_2 \not\in \mathbb{R}} }{ \eval(\texttt{Plus(e1, e2)}, \sigma, s) = (\mathbf{error} , \color{blue}{s_2})}{plus-nok-2}
$$

#### Some of the other rules we will need:

If-then-else:
$$\semRule{\eval(\texttt{e}, \sigma, s) = (true, \color{red}{s_1})\;\;\; 
\eval(\texttt{e1}, \sigma, \color{red}{s_1}) = (v_1, \color{blue}{s_2}) }{ \eval(\texttt{IfThenElse(e, e1, e2)}, \sigma, s) = (v_1, \color{blue}{s_2})}{ite-then}$$


$$\semRule{\eval(\texttt{e}, \sigma, s) = (false, \color{red}{s_1})\;\;\;
\eval(\texttt{e2}, \sigma, \color{red}{s_1}) = (v_2, \color{blue}{s_2}) }{ \eval(\texttt{IfThenElse(e, e1, e2)}, \sigma, s) = (v_2, \color{blue}{s_2})}{ite-else}$$

$$\semRule{\eval(\texttt{e}, \sigma, s) = (b, \color{red}{s_1})\;\;\; b \not\in \mathbb{B}}{ \eval(\texttt{IfThenElse(e, e1, e2)}, \sigma, s) = (\mathbf{error}, \color{red}{s_1})}{ite-nok}$$

Lets: 
$$\semRule{ \eval(\texttt{e1}, \sigma, s) = (v_1, \color{red}{s'})\;\; v_1 \not= \mathbf{error} }{ \eval(\texttt{Let(x, e1, e2)}, \sigma, s) = \eval(\texttt{e2}, \sigma[x \mapsto v_1] , \color{red}{s'}) }{let-ok}$$

Functions:

$$\semRule{}{\eval(\texttt{FunDef(x, e)}, \sigma, s) = (\texttt{Closure}(x, e, \sigma), s )}{fundef}$$

$$\semRule{\eval(\texttt{e1}, \sigma, s) = (\texttt{Closure}(x, \texttt{fbody}, \pi), s_1) \;\;\; \eval(\texttt{e2}, \sigma, s_1) = (v, s_2),\;\; v \not= \mathbf{error} }{
\eval(\texttt{FunCall(e1, e2)}, \sigma, s) = \eval(\texttt{fbody}, \pi[x \mapsto v], s_2) }{funcall-ok}$$

#### Operations on References

Let us start with NewRef

$$\semRule{\eval(\texttt{e}, \sigma, s) = (v, s_1), \;\; v \not= \mathbf{error},\;\;\; \texttt{createNewCell}(s_1,v) = (j, s_2) }{\eval(\texttt{NewRef(e)}, \sigma, s) = (\texttt{Reference}(j), s_2) }{newref-ok}$$
Let's break down this rule.

Next, let us do DeRef. 

$$\semRule{\eval(\texttt{e}, \sigma, s) = (r, s_1), \;\; r = \texttt{Reference}(j)\;\; \texttt{lookupCell}(s_1, j) = v }{\eval(\texttt{Deref(e)}, \sigma, s) = (v, s_1) }{deref-ok}$$

Finally, we can do assignment


$$\semRule{\eval(\texttt{e1}, \sigma, s) = (r, s_1), \;\; r = \texttt{Reference}(j)\;\; \eval(\texttt{e2}, \sigma, s_1) = (v_2, s_2), \;\; \texttt{assignToCell}(s_2, j, v_2) = s_3 }{\eval(\texttt{AssignRef(e1, e2)}, \sigma, s) = (v_2, s_3) }{assign-ref-ok}$$

In [ ]:
sealed trait Value


/*-- Now we can finish the rest --*/
case class NumValue(f: Double) extends Value
case class BoolValue(b: Boolean) extends Value
/*-- Note: to get recursion working, we will need to make environments different --*/
case class Closure(x: String, e: Expr, pi: Map[String, Value]) extends Value 
/* -- references are here -- */
case class Reference(j: Int) extends Value
case object ErrorValue extends Value


/*2. Operators on values */

def valueToNumber(v: Value): Double = v match {
    case NumValue(d) => d
    case _ => throw new IllegalArgumentException(s"Error: Asking me to convert Value: $v to a number")
}

def valueToBoolean(v: Value): Boolean = v match {
    case BoolValue(b) => b
    case _ => throw new IllegalArgumentException(s"Error: Asking me to convert Value: $v to a boolean")
}

def valueToClosure(v: Value): Closure = v match {
    case Closure(x, e, pi) => Closure(x, e, pi)
    case _ =>  throw new IllegalArgumentException(s"Error: Asking me to convert Value: $v to a closure")
}

/*3. Immutable Store */

case class ImmutableStore(val nCells: Int, val storeMap: Map[Int, Value])
    
def createNewCell(s: ImmutableStore, v: Value): (ImmutableStore, Int) = {
        /*- make a new cell -*/
        val j = s.nCells
        val nMap = s.storeMap + (j -> v)
        val nStore = ImmutableStore(s.nCells + 1, nMap) // Make a new store with one more cell
        (nStore, j)
}
    
def lookupCellValue(s: ImmutableStore, j: Int): Value = {
        if (s.storeMap.contains(j)){
            s.storeMap(j)
        } else {
            throw new IllegalArgumentException(s"Illegal lookup of nonexistant location $j")
        }
}
    
def assignToCell(s: ImmutableStore, j: Int, v: Value): ImmutableStore = {
        if (s.storeMap.contains(j)){
            val nMap = s.storeMap + (j -> v) // Update the store map.
            ImmutableStore(s.nCells, nMap)
        } else {
            throw new IllegalArgumentException(s"Illegal assignment to nonexistent location $j")
        }
    }

In [ ]:
def evalExpr(e: Expr, env: Map[String, Value], store: ImmutableStore): (Value, ImmutableStore) = {
      /* Method to deal with binary arithmetic operations */
    
    def applyArith2 (e1: Expr, e2: Expr) (fun: (Double , Double) => Double) = {
        ???
    }  /* -- We have deliberately curried the method --*/
    
    /* Helper method to deal with unary arithmetic */
    def applyArith1(e: Expr) (fun: Double => Double) = {
        val (v,store1) = evalExpr(e, env, store)
        val v1 = fun(valueToNumber(v))
        (NumValue(v1), store1)
    }
    
    /* Helper method to deal with comparison operators */
    def applyComp(e1: Expr, e2: Expr) (fun: (Double, Double) => Boolean) = {
        val (v1, store1) = evalExpr(e1, env, store)
        val (v2, store2) = evalExpr(e2, env, store1)
        val v3 = fun(valueToNumber(v1), valueToNumber(v2))
        (BoolValue(v3), store2)
    }
    
    e match {
        case Const(f) => (NumValue(f), store)
        
        case Ident(x) => {
            if (env contains x) 
                (env(x), store)
            else 
                throw new IllegalArgumentException(s"Undefined identifier $x")
        }
    
    
        case Plus(e1, e2) => applyArith2 (e1, e2) ( _ + _ )
            
        case Minus(e1, e2) => applyArith2(e1, e2) ( _ - _ )
    
        case Mult(e1, e2) =>  applyArith2(e1, e2) (_ * _)
        
        case Geq(e1, e2) => applyComp(e1, e2)(_ >= _)
    
        case Eq(e1, e2) => applyComp(e1, e2)(_ == _)
        
        case IfThenElse(e1, e2, e3) => {
            val (v, store1) = evalExpr(e1, env, store)
            v match {
                case BoolValue(true) => evalExpr(e2, env, store1)
                case BoolValue(false) => evalExpr(e3, env, store1)
                case _ => throw new IllegalArgumentException(s"If-then-else condition expr: ${e1} is non-boolean -- evaluates to ${v}")
            }
        }
        
        case Let(x, e1, e2) => {
            val (v1, store1) = evalExpr(e1, env, store)  // eval e1
            val env2 = env + (x -> v1) // create a new extended env
            evalExpr(e2, env2, store1) // eval e2 under that.
        }
    
        case FunDef(x, e) => {
            (Closure(x, e, env), store) // Return a closure with the current enviroment.
        }
        
        case FunCall(e1, e2) => {
            val (v1, store1) = evalExpr(e1, env, store)
            val (v2, store2) = evalExpr(e2, env, store1)
            v1 match {
                case Closure(x, closure_ex, closed_env) => {
                    // First extend closed_env by binding x to v2
                    val new_env = closed_env + ( x -> v2)
                    // Evaluate the body of the closure under the extended environment.
                    evalExpr(closure_ex, new_env, store2)
                }
                case _ => throw new IllegalArgumentException(s"Function call error: expression $e1 does not evaluate to a closure")
            }
        }
        
        case NewRef(e) => {
            val (v, store1) = evalExpr(e, env, store)
            val (store2, j) = createNewCell(store1, v)
            (Reference(j), store2)
        }
        
        case DeRef(e) => {
            val (v, store1) = evalExpr(e, env, store)
            v match {
                case Reference(j) => {
                    val v = lookupCellValue(store1, j)
                    (v, store1)
                }
                case _ => throw new IllegalArgumentException(s"Deref applied to an expression that does not evaluate to a reference")
            }
        }
        
        case AssignRef(e1, e2) => {
            val (v1, store1) = evalExpr(e1, env, store)
            v1 match {
                case Reference(j) => {
                    val (v2, store2) = evalExpr(e2, env, store1)
                    val store3 = assignToCell(store2, j, v2)
                    (v2, store3)// Return the  assigned value
                }
                case _ => throw new IllegalArgumentException(s"AssignRef applied to argument that is not a reference")
                
            }
        }
        
    }

}

def evalProgram(p: Program) = p match {
        case TopLevel(e) => { 
            // Start with empty environment and empty store
            val (v1, s1) = evalExpr(e, Map(), new ImmutableStore(0, Map()))
            v1
        }
}
    

In [ ]:
/*
let x = NewRef(10) in 
let y = DeRef(x) + 1 in 
let z = AssignRef(x, y) in 
   z*/

val z = Ident("z")
val x = Ident("x")
val y = Ident("y")

val e1 = Let("z", AssignRef(x, y), z)
val e2 = Let("y", Plus(Const(1.0), DeRef(x)), e1)
val e = Let("x", NewRef(Const(10.0)), e2)
val prog = TopLevel(e)

println(s"Result: ${evalProgram(prog)}")

## Implicit References

Most languages support for mutable state is very implicit: we do not have to do or say anything to get mutable variables, they just are mutable.
Assigning to a variable works, and using the value works.

In particular, having to declare the mutable variables using refs, and having to dereference when we want to use the value can be cumbersome (and error prone), so we would like to do away with that.

Think variables in most languages, like Java, Scala, Python, etc.

We can add support to Lettuce similar to Scala's support as follows:

$$\begin{array}{rcll}
\mathbf{Program} & \rightarrow & TopLevel(\mathbf{Expr}) \\[5pt]
\mathbf{Expr} & \rightarrow & Const(\mathbf{Number}) \\
 & | & Ident(\mathbf{Identifier}) \\
 & | & Plus(\mathbf{Expr}, \mathbf{Expr}) \\
 & | & Minus(\mathbf{Expr}, \mathbf{Expr}) \\
 & | & Mult (\mathbf{Expr}, \mathbf{Expr}) \\
 & | & Geq (\mathbf{Expr}, \mathbf{Expr}) \\
 & | & Eq (\mathbf{Expr}, \mathbf{Expr}) \\
 & | & IfThenElse(\mathbf{Expr}, \mathbf{Expr}, \mathbf{Expr}) & \text{if (expr) then expr else expr} \\
 & | & Let( \mathbf{Identifier}, \mathbf{Expr}, \mathbf{Expr}) & \text{let identifier = expr in expr} \\
 & | & FunDef( \mathbf{Identifier}, \mathbf{Expr}) & \text{function (identifier-formal-parameter) expr } \\ 
 & | & FunCall(\mathbf{Expr}, \mathbf{Expr}) & \text{function call - expr(expr)} \\
 & | & \color{red}{LetVar}(\mathbf{Identifier}, \mathbf{Expr}, \mathbf{Expr}) & \text{let var stmt -- compare to let binding.}\\
 & | & \color{red}{AssignVar}(\mathbf{Identifier}, \mathbf{Expr}) & \text{assign a var to a value. }
\end{array}$$


In [ ]:
sealed trait Program
sealed trait Expr

case class TopLevel(e: Expr) extends Program

case class Const(v: Double) extends Expr // Expr -> Const(v)
case class Ident(s: String) extends Expr // Expr -> Ident(s)

// Arithmetic Expressions
case class Plus(e1: Expr, e2: Expr) extends Expr // Expr -> Plus(Expr, Expr)
case class Minus(e1: Expr, e2: Expr) extends Expr // Expr -> Minus(Expr, Expr)
case class Mult(e1: Expr, e2: Expr) extends Expr // Expr -> Mult (Expr, Expr)

// Boolean Expressions
case class Geq(e1: Expr, e2:Expr) extends Expr
case class Eq(e1: Expr, e2: Expr) extends Expr

//If then else
case class IfThenElse(e: Expr, eIf: Expr, eElse: Expr) extends Expr

//Let bindings
case class Let(s: String, defExpr: Expr, bodyExpr: Expr) extends Expr

//Function definition
case class FunDef(param: String, bodyExpr: Expr) extends Expr

// Function call
case class FunCall(funCalled: Expr, argExpr: Expr) extends Expr

// Let Var
case class LetVar(x: String, e1: Expr, e2: Expr) extends Expr

// Assign Var
case class AssignVar(x: String, e: Expr) extends Expr

We can provide support for implicit references in the same way as we did for explicit references: using a store and representing references as addresses in the store.

This leads us to the inference rules for the implicit references:
The rules that do not contain references, such as constants, addition, and if-then-else, are left unchanged from the explicit references case.

Our different rules are the identifier case, and the new rules for LetVar and AssignVar:

The following rule ensures that whenever we evaluate an identifier $x$ that happens to be
a var, we will actually return the stored value.
$$\semRule{x \in \text{domain}(\sigma),\ \sigma(x) = \texttt{Reference}(j), \texttt{lookupCell}(s, j) = v}{ \eval(\texttt{Ident(x)}, \sigma, s) = (v, s) }{ident-var-ok}$$

For immutable, we have the usual semantics:

$$\semRule{x \in \text{domain}(\sigma),\ \sigma(x) = v, v \not= \texttt{Reference}(j) }{ \eval(\texttt{Ident(x)}, \sigma, s) = (v, s) }{ident-val-ok}$$

We will now write the semantic rule for let ref. The idea is that we will generate a new reference.

$$\semRule{\eval(\texttt{e1}, \sigma, s) = (v, s_1), \;\; v \not= \mathbf{error},\;\;\; \texttt{createNewCell}(s_1,v) = (j, s_2)\;\;  }{\eval(\texttt{LetVar(x, e1, e2)}, \sigma, s) =  \eval(\texttt{e2}, \sigma[x \mapsto \texttt{Reference}(j)], s_2 )}{let-var-ok}$$

The semantic rule for `AssignVar` is the same as that for `AssignRef` but under a new guise.

$$\semRule{x \in \text{domain}(\sigma),\ \sigma(x) = \texttt{Reference}(j),\;\;\eval(\texttt{e}, \sigma, s) = (v, s_1), \;\;  \texttt{assignToCell}(s_1, j, v) = s_2}{ \eval(\texttt{AssignVar(x, e)}, \sigma, s) = (v, s_2) } {assign-var-ok} $$

In [ ]:
// Copy from the case for explicit references
sealed trait Value


/*-- Now we can finish the rest --*/
case class NumValue(f: Double) extends Value
case class BoolValue(b: Boolean) extends Value
/*-- Note: to get recursion working, we will need to make environments different --*/
case class Closure(x: String, e: Expr, pi: Map[String, Value]) extends Value 
/* -- references are here -- */
case class Reference(j: Int) extends Value
case object ErrorValue extends Value


/*2. Operators on values */

def valueToNumber(v: Value): Double = v match {
    case NumValue(d) => d
    case _ => throw new IllegalArgumentException(s"Error: Asking me to convert Value: $v to a number")
}

def valueToBoolean(v: Value): Boolean = v match {
    case BoolValue(b) => b
    case _ => throw new IllegalArgumentException(s"Error: Asking me to convert Value: $v to a boolean")
}

def valueToClosure(v: Value): Closure = v match {
    case Closure(x, e, pi) => Closure(x, e, pi)
    case _ =>  throw new IllegalArgumentException(s"Error: Asking me to convert Value: $v to a closure")
}

/*3. Immutable Store -- unoptimized impl. */

case class ImmutableStore(val nCells: Int, val storeMap: Map[Int, Value])
    
def createNewCell(s: ImmutableStore, v: Value): (ImmutableStore, Int) = {
        /*- make a new cell -*/
        val j = s.nCells
        val nMap = s.storeMap + (j -> v)
        val nStore = ImmutableStore(s.nCells + 1, nMap) // Make a new store with one more cell
        (nStore, j)
}
    
def lookupCellValue(s: ImmutableStore, j: Int): Value = {
        if (s.storeMap.contains(j)){
            s.storeMap(j)
        } else {
            throw new IllegalArgumentException(s"Illegal lookup of nonexistant location $j")
        }
}
    
def assignToCell(s: ImmutableStore, j: Int, v: Value): ImmutableStore = {
        if (s.storeMap.contains(j)){
            val nMap = s.storeMap + (j -> v) // Update the store map.
            ImmutableStore(s.nCells, nMap)
        } else {
            throw new IllegalArgumentException(s"Illegal assignment to nonexistent location $j")
        }
    }
    

In [ ]:
def evalExpr(e: Expr, env: Map[String, Value], store: ImmutableStore): (Value, ImmutableStore) = {
      /* Method to deal with binary arithmetic operations */
    
    def applyArith2 (e1: Expr, e2: Expr) (fun: (Double , Double) => Double) = {
        val (v1, store1) = evalExpr(e1, env, store)
        val (v2, store2) = evalExpr(e2, env, store1)
        val v3 = fun(valueToNumber(v1), valueToNumber(v2))
        (NumValue(v3), store2)
    }  /* -- We have deliberately curried the method --*/
    
    /* Helper method to deal with unary arithmetic */
    def applyArith1(e: Expr) (fun: Double => Double) = {
        val (v,store1) = evalExpr(e, env, store)
        val v1 = fun(valueToNumber(v))
        (NumValue(v1), store1)
    }
    
    /* Helper method to deal with comparison operators */
    def applyComp(e1: Expr, e2: Expr) (fun: (Double, Double) => Boolean) = {
        val (v1, store1) = evalExpr(e1, env, store)
        val (v2, store2) = evalExpr(e2, env, store1)
        val v3 = fun(valueToNumber(v1), valueToNumber(v2))
        (BoolValue(v3), store2)
    }
    
    e match {
        case Const(f) => (NumValue(f), store)
        
        case Ident(x) => {
            if (env contains x ) { // In scala a.b(c) can simply be written as "a b c" <macro>
                val v = env(x)
                v match {
                    case Reference(j) => { // AUTO deref
                        ???
                    }
                    case _ => ???
                }  
            } else 
                throw new IllegalArgumentException(s"Undefined identifier $x")
        }
    
    
        case Plus(e1, e2) => applyArith2 (e1, e2) ( _ + _ )
            
        case Minus(e1, e2) => applyArith2(e1, e2) ( _ - _ )
    
        case Mult(e1, e2) =>  applyArith2(e1, e2) (_ * _)
        
        case Geq(e1, e2) => applyComp(e1, e2)(_ >= _)
    
        case Eq(e1, e2) => applyComp(e1, e2)(_ == _)
        
        case IfThenElse(e1, e2, e3) => {
            val (v, store1) = evalExpr(e1, env, store)
            v match {
                case BoolValue(true) => evalExpr(e2, env, store1)
                case BoolValue(false) => evalExpr(e3, env, store1)
                case _ => throw new IllegalArgumentException(s"If-then-else condition expr: ${e1} is non-boolean -- evaluates to ${v}")
            }
        }
        
        case Let(x, e1, e2) => {
            val (v1, store1) = evalExpr(e1, env, store)  // eval e1
            val env2 = env + (x -> v1) // create a new extended env
            evalExpr(e2, env2, store1) // eval e2 under that.
        }
    
        case FunDef(x, e) => {
            (Closure(x, e, env), store) // Return a closure with the current enviroment.
        }
        
        case FunCall(e1, e2) => {
            val (v1, store1) = evalExpr(e1, env, store)
            val (v2, store2) = evalExpr(e2, env, store1)
            v1 match {
                case Closure(x, closure_ex, closed_env) => {
                    // First extend closed_env by binding x to v2
                    val new_env = closed_env + ( x -> v2)
                    // Evaluate the body of the closure under the extended environment.
                    evalExpr(closure_ex, new_env, store2)
                }
                case _ => throw new IllegalArgumentException(s"Function call error: expression $e1 does not evaluate to a closure")
            }
        }
        
        
        
        case AssignVar(x, e) => { // x is a string -- name of identifier and e is Expr -- RHS of assignment
            val (v1, store1) = evalExpr(e, env, store) // First evaluate e
            val v2 =  if (env contains x)  // Next, check x from the current environment
                        env(x)
                      else 
                        throw new IllegalArgumentException(s"Undefined identifier $x")// Trying to assign to an undeclared identifier
            v2 match {
                case Reference(j) => { // x better be a reference in the current env.
                    val store3 = assignToCell(store1, j, v1) // assign to cell function in ImmutableStore API
                    (v1, store3) 
                }
                case _ => throw new IllegalArgumentException(s"AssignVar applied to argument that is not a mutable var")
                
            }
        }
        
        case LetVar(x, e1, e2) => { // let var x = e1 in e2 
            // This is the same treatment as let x = newref(e1) in e2 in ExplicitRef Language.
            val (v1, store1) = evalExpr(e1, env, store) // evaluate e1
            val (store2, j) = createNewCell(store1, v1) // create a new cell corresponding to the value of e1
            val newEnv = env + (x -> Reference(j)) // update the environment
            evalExpr(e2, newEnv, store2) // evaluatet e2 with the new environment and the new store.
        }
        
    }

}

def evalProgram(p: Program) = p match {
        case TopLevel(e) => { 
            // Start with empty environment and empty store
            val (v1, s1) = evalExpr(e, Map(), new ImmutableStore(0, Map()))
            v1
        }
}
    


In [ ]:
/* let var x = 10 in 
    let dummy = AssignVar(x, 20) in 
        x
        */
val x = Ident("x")
val e1 = Let("dummy", AssignVar("x", Const(20)), x)
val e2 = LetVar("x", Const(10), e1)
val prog = TopLevel(e2)

println(s"Result = ${evalProgram(prog)}")
